## Ανάλυση IMDB δεδομένων
### Random Forest Ταξινομητής

Ορίζουμε τις **υπερπαραμέτρους** που θα χρησιμοποιηθούν για την ανάπλαση των δωσμένων δεδομένων.

In [1]:
m = 3000   # Πλήθος λέξεων του λεξιλογίου
n = 50     # Πιο συχνές λέξεις για παράληψη
k = 1000   # Λιγότερο συχνές λέξεις για παράληψη

Αντλούμε τα δεδομένα από τη βάση δεδομένων IMDB, αγνοώντας τις πιο συχνά χρησιμοποιούμενες λέξεις n και τις λιγότερες χρησιμοποιούμενες λέξεις k. (υπερπαράμετροι)

In [2]:
import tensorflow as tf
import numpy as np
from math import log 

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=m-k, skip_top=n)
word_index = tf.keras.datasets.imdb.get_word_index()

index2word = dict((i + 3, word) for (word, i) in word_index.items())
index2word[0] = '[pad]'
index2word[1] = '[bos]'
index2word[2] = '[oov]'

x_train = np.array([' '.join([index2word[idx] for idx in text]) for text in x_train])
x_test = np.array([' '.join([index2word[idx] for idx in text]) for text in x_test])

### Δημιουργία δυαδικών διανυσμάτων

Μετατρέπουμε τα αντλημένα δεδομένα σε δυαδικά διανύσματα. Οι δυνατές τιμές είναι 0 και 1. Τιμή 1 συνεπάγεται ότι η αντίστοιχη λέξη περιέχεται στο κείμενο, ενώ τιμή 0 συνεπάγεται ότι δεν περιέχεται.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

binary_vectorizer = CountVectorizer(binary=True)
x_train_binary = binary_vectorizer.fit_transform(x_train)
x_test_binary = binary_vectorizer.transform(x_test)

x_train_binary = np.array(x_train_binary.toarray())
x_test_binary = np.array(x_test_binary.toarray())
print(
    'Vocabulary size:', len(binary_vectorizer.vocabulary_)
)

Vocabulary size: 1906


Χ είναι ένας πίνακας διανυσμάτων (αξιολογήσεων), με κάθε διάνυσμα να αναπαριστά λέξεις. Y είναι ένα διάνυσμα με ετικέτες (0 ή 1) όπου 0 είναι μια αρνητική αξιολόγηση και 1 μια θετική.

$$X = \begin{bmatrix} \vec{x_{1}} \\ \vdots \\ \vec{x_{m}} \end{bmatrix}\, \, \, 
y = \begin{bmatrix} y_{1} \\ \vdots \\ y_{m} \end{bmatrix}$$

### Υλοποίηση του Random Forest Ταξινομητή

Κλάση που υλοποιεί τον Random Forest Ταξινομητή. Η κλαση αποτελείται από δύο βασικές μεθόδους, την fit και την predict. Η μέθοδος fit εκπαιδεύει τον αλγόριθμο χρησιμόποιώντας τον πίνακα των δυαδικών διανυσμάτων X και τον πίνακα με τις ετικέτες y. Η μέθοδος predict δέχεται ένα πίνακα διανυσμάτων και επιστρέφει ένα προβλεπόμενο διάνυσμα με ετικέτες.

In [ ]:
from statistics import mode
import numpy as np
from id3 import ID3

class RandomForest:
    def __init__(self, m, num_of_trees, max_depth=5):
        self.m = m
        self.num_of_trees = num_of_trees
        self.max_depth = max_depth
        self.random_trees = []

    def fit(self, X, y):
        self.random_trees.clear()
        print("Starting fitting process")

        for i in range(self.num_of_trees):
            print(f"Fitting tree {i + 1}/{self.num_of_trees}")
            random_x, random_y = self.select_random_samples(X, y)

            if len(random_x) > 0:
                print(f"Random samples: {random_x.shape}")
                random_features = self.select_random_features(random_x)
                id3 = ID3(random_features)#self.max_depth)
                tree = id3.fit(random_x, random_y)
                
                if tree is not None:  # Check if a valid tree is returned
                    print(f"Tree {i + 1} fitted successfully")
                    self.random_trees.append(id3)
                else:
                    print(f"Skipping tree {i + 1} due to empty data or other issues.")
            else:
                print(f"Skipping tree {i + 1} due to empty data.")

        print("Fitting process completed")

    def select_random_samples(self, X, y):
        x_sample = list()
        y_sample = list()
        indices = np.arange(len(y))

        for i in range(len(y)):
            random_choice = np.random.choice(indices)
            x_sample.append(X[random_choice])
            y_sample.append(y[random_choice])

        x_sample = np.array(x_sample)
        y_sample = np.array(y_sample)   

        return x_sample, y_sample

    def select_random_features(self, random_x):
        indices = np.arange(len(random_x[0]))
        random_feature_indices = np.random.choice(indices, self.m, replace=False)

        return random_feature_indices
    
    def predict(self, X):
        predictions = []
        total = list()
        
        for tree in self.random_trees:
            categories = tree.predict(X)
            total.append(np.array(categories))

        for i in range(0, X.shape[0]):################ 
            num1 = 0
            num0 = 0
            for category in total:
                if(category[i] == 1):
                    num1 +=1
                else:
                    num0 +=1
            if num1 > num0:
                predictions.append(1)
            else:
                predictions.append(0)

        return np.array(predictions)

In [ ]:
from visualizations import *

In [ ]:
data = classification_data(RandomForest(200, num_of_trees = 3), x_train_binary, y_train, x_test_binary, y_test, 5)

print(classification_report(y_test, data['test_predictions']))
data_table = classification_table(data, x_train_binary)
ipd.display(data_table)

In [ ]:
rf = RandomForest(200, num_of_trees = 3)
rf.fit(x_train_binary, y_train)
print(classification_report(y_train, rf.predict(x_train_binary),
                            zero_division=1))
print(classification_report(y_test, rf.predict(x_test_binary),
                            zero_division=1))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion='entropy', max_depth=5, max_features=200)
rf.fit(x_train_binary, y_train)
print(classification_report(y_train, rf.predict(x_train_binary),
                            zero_division=1))
print(classification_report(y_test, rf.predict(x_test_binary),
                            zero_division=1))